# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
%matplotlib inline

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# load and Read CSV File
weather_data = pd.read_csv('../output_data/weather_data.csv', index_col = 0)
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Dikson,73.51,80.55,-3.97,94,98,7.26,RU,1603165638
1,Rikitea,-23.12,-134.97,23.47,77,0,5.23,PF,1603165589
2,San Cristobal,7.77,-72.22,19.54,83,95,0.69,VE,1603165631
3,Ushuaia,-54.80,-68.30,2.00,89,75,15.40,AR,1603165556
4,Busselton,-33.65,115.33,25.00,33,5,2.91,AU,1603165642
...,...,...,...,...,...,...,...,...,...
574,Gigmoto,13.78,124.39,27.80,82,100,5.99,PH,1603166313
575,Rialma,-15.31,-49.58,21.69,87,98,1.44,BR,1603166315
576,Anadyr,64.75,177.48,3.00,85,40,5.00,RU,1603166316
577,Saint Stephen,50.33,-4.90,14.00,93,75,8.70,GB,1603166317


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Check Data type
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 579 entries, 0 to 578
Data columns (total 9 columns):
City          579 non-null object
Lat           579 non-null float64
Lng           579 non-null float64
Max Temp      579 non-null float64
Humidity      579 non-null int64
Cloudiness    579 non-null int64
Wind Speed    579 non-null float64
Country       576 non-null object
Date          579 non-null int64
dtypes: float64(4), int64(3), object(2)
memory usage: 45.2+ KB


In [4]:
# Change Humidity data type from integer to float
humidity = weather_data["Humidity"].astype(float)
humidity.head()

0    94.0
1    77.0
2    83.0
3    89.0
4    33.0
Name: Humidity, dtype: float64

In [5]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

-----
### Plotting Heatmap

In [6]:
# Create a Heatmap layer
fig = gmaps.figure( display_toolbar = True,
                    display_errors = True,
                    zoom_level = 2,
                    tilt = 45,
                    center = (0,30),
                    layout = None,
                    map_type = 'ROADMAP',
                    mouse_handling ='COOPERATIVE', )

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights = humidity, 
                                 dissipating = False, 
                                 max_intensity = 100,
                                 point_radius = 3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

-----

### Create new DataFrame fitting weather criteria

* Narrow down the DataFrame to find your ideal weather condition. For example:

* A max temperature lower than 80 degrees but higher than 70.

* Wind speed less than 10 mph.

* Zero cloudiness.

* Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.* 

In [7]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 70 and 80
# since Max Temp measurment is in celcius:changed the temps from Fahrenheit to Celcius for below formula
# (80°F − 32) × 5/9 = 26.667°C
# (70°F − 32) × 5/9 = 21.111°C
ideal_weather = (weather_data.loc[(weather_data["Max Temp"] < 26.66) & 
                                  (weather_data["Max Temp"] > 21.11) & 
                                  (weather_data["Wind Speed"] < 10) & 
                                  (weather_data["Cloudiness"] == 0)])

# Drop any rows that don't contain all 4 conditions
ideal_weather = ideal_weather.dropna(how = 'any')


# # Reset Dataframe index
ideal_weather.reset_index()

# Display DataFrame
ideal_weather


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,Rikitea,-23.12,-134.97,23.47,77,0,5.23,PF,1603165589
21,Hong Kong,22.29,114.16,26.11,64,0,0.89,HK,1603165437
43,Yorosso,12.35,-4.78,24.27,67,0,1.88,ML,1603165688
179,Ţahţā,26.77,31.50,22.00,37,0,2.60,EG,1603165848
184,Tambacounda,13.75,-13.76,25.00,88,0,1.00,SN,1603165854
212,Mitsamiouli,-11.38,43.28,25.57,70,0,1.72,KM,1603165825
278,Aswān,24.09,32.91,25.09,25,0,3.72,EG,1603165966
341,Cap Malheureux,-19.98,57.61,25.56,67,0,5.36,MU,1603166040
380,Longhua,23.61,114.19,26.06,51,0,2.67,CN,1603166085
434,Bandar-e Lengeh,26.56,54.88,22.00,64,0,3.10,IR,1603166149


### Export & Save Data
* Export the city data into a .csv.

In [8]:
# Export DataFrame & Save DataFrame it to a .CSV
ideal_weather.to_csv('../output_data/ideal_weather.csv')

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.

In [9]:
# Creat a new variable for hotel_df
hotel_df = ideal_weather.loc[:, ["City", "Country", "Lat", "Lng"]]
hotel_df ["Hotel Name"] = ""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
1,Rikitea,PF,-23.12,-134.97,
21,Hong Kong,HK,22.29,114.16,
43,Yorosso,ML,12.35,-4.78,
179,Ţahţā,EG,26.77,31.50,
184,Tambacounda,SN,13.75,-13.76,


* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Use the search term: "Hotel" and our lat/lng
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify cities
for index, row in hotel_df.iterrows():
    # get lat, lng, city from df
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # make API request and convert to json
    response = requests.get(base_url, params=params).json()
    #pretty print json
    #pprint(response)
    
    # extract results
    results = response['results']

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        print()
    
    # if there is no hotel available don't stop the loop pring "Missing field/result.. Skipping" and go to next row
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    print("------------")
              
    # Wait a second in loop to not over exceed rate limit of API
    time.sleep(1)

# Print Ending of search result              
print(f"-------------------------------")
print(f"Search Completed")
print(f"-------------------------------")
 

Closest hotel in Rikitea is Pension Maro'i.

------------
Closest hotel in Hong Kong is Royal Plaza Hotel 帝京酒店.

------------
Missing field/result... skipping.
------------
Closest hotel in Ţahţā is El Salam Hotel.

------------
Missing field/result... skipping.
------------
Closest hotel in Mitsamiouli is Al Camar Lodge.

------------
Closest hotel in Aswān is Sofitel Legend Old Cataract Aswan.

------------
Closest hotel in Cap Malheureux is Paradise Cove Boutique Hotel.

------------
Missing field/result... skipping.
------------
Closest hotel in Bandar-e Lengeh is Diplomat Hotel.

------------
Closest hotel in Álamos is Hacienda De Los Santos Resort & Spa.

------------
Missing field/result... skipping.
------------
Closest hotel in Gusau is Green Scenery Hotel.

------------
Closest hotel in Maiduguri is AMADA INTERNATIONAL HOTEL.

------------
Closest hotel in Bama is Borno State Hotel.

------------
-------------------------------
Search Completed
-------------------------------

#### Export & Save Data
* Export the Hotel data into a .csv

In [11]:
# Export DataFrame & Save DataFrame it to a .CSV
hotel_df.to_csv('../output_data/hotel_df.csv')

# Display the DataFrame Head
hotel_df

,City,Country,Lat,Lng,Hotel Name
1,Rikitea,PF,-23.12,-134.97,Pension Maro'i
21,Hong Kong,HK,22.29,114.16,Royal Plaza Hotel 帝京酒店
43,Yorosso,ML,12.35,-4.78,
179,Ţahţā,EG,26.77,31.50,El Salam Hotel
184,Tambacounda,SN,13.75,-13.76,
212,Mitsamiouli,KM,-11.38,43.28,Al Camar Lodge
278,Aswān,EG,24.09,32.91,Sofitel Legend Old Cataract Aswan
341,Cap Malheureux,MU,-19.98,57.61,Paradise Cove Boutique Hotel
380,Longhua,CN,23.61,114.19,
434,Bandar-e Lengeh,IR,26.56,54.88,Diplomat Hotel


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer and info box ontop of heat map
markers = gmaps.marker_layer(locations,
                             hover_text='',
                             label='',
                             info_box_content=hotel_info,
                             display_info_box=None,
)


# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))